# Model

1. Dividir el set de datos entre valores dependientes (y) e independientes (X). Y sería la columna
que se quiere predecir (“churn”).
2. Generar sets de testeo y entrenamiento.
3. ¿Es necesario escalar las features? Hacerlo si fuera necesario.
4. Probar por lo menos dos modelos y seleccionar uno. Explicar porque lo selecciono y qué
métricas uso para decidir.

<hr/>

In [63]:
# Libraries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.close("all")

import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm


In [64]:
df_churn = pd.read_csv('../data/preprocessed/churn_preprocessed.csv')

In [65]:
df_churn.dtypes

Unnamed: 0.1          int64
Unnamed: 0            int64
customerID           object
gender               object
SeniorCitizen        object
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [66]:
df_churn.sample(5)

,Unnamed: 0.1,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
1116,2120,2120,5793-YOLJN,Female,No,Yes,Yes,55.0,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,21.00,1210.30,No
347,347,347,8966-SNIZF,Female,No,Yes,No,70.0,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),19.45,1303.50,No
3097,4103,4103,5214-NLTIT,Male,No,Yes,Yes,72.0,Yes,Yes,...,Yes,No,No,Yes,Two year,Yes,Credit card (automatic),90.80,6511.80,No
5619,6628,6628,9979-RGMZT,Female,No,No,No,7.0,Yes,No,...,No,No,Yes,Yes,One year,Yes,Mailed check,94.05,633.45,No
207,207,207,1285-OKIPP,Male,No,No,No,1.0,Yes,No,...,No,No,No,Yes,Month-to-month,No,Electronic check,79.90,79.90,Yes


## Features selection

In [67]:
df_churn = df_churn.drop(['Unnamed: 0.1', 'Unnamed: 0', 'customerID'], axis=1)

### Train-test split

In [68]:
X = df_churn.drop('Churn', axis=1)
y = df_churn.Churn

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=12)

Churn = Yes shows same proportion of the dataframe before and after split x, y, train and test. It means that stratify= y is keeping proportions through dataframe splitting process.

In [70]:
df_churn.Churn.value_counts()['Yes'] / df_churn.shape[0]

0.26492042440318303

In [71]:
y_train.value_counts()['Yes'] / y_train.shape[0]

0.26480990274093724

In [72]:
y_test.value_counts()['Yes'] / y_test.shape[0]

0.26525198938992045

Dummies

In [73]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)
X_train.sample(5)

,tenure,MonthlyCharges,TotalCharges,gender_Male,SeniorCitizen_Yes,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
2867,51.0,34.20,1782.00,0,0,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4154,8.0,19.45,159.20,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,0,0,0
2089,28.0,54.40,1516.60,1,0,0,1,1,0,0,...,0,0,0,0,1,0,1,1,0,0
3149,72.0,117.35,8436.25,0,1,0,0,1,0,1,...,0,1,0,1,0,1,1,1,0,0
2494,1.0,20.90,20.90,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,1


## Model

In [74]:
# Instanciado del modelo

model_1 = LogisticRegression(penalty='none') # sin regularización
model_1.fit(X_train, y_train);

/home/gera/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/gera/miniconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [75]:
y_test_pred_1 = model_1.predict(X_test)
y_test_pred_1

array(['No', 'Yes', 'No', ..., 'No', 'No', 'Yes'], dtype=object)

El método .predict_proba() devuelve un array con dos probabilidades para cada instancia del test set: 
p(y=0) y p(y=1), en ese orden.

La primera columna es la probabilidad de pertenecer a la clase 0 (negativa), y la segunda columna a la clase 1 (positiva).

In [76]:
y_test_pred_proba = model_1.predict_proba(X_test)
y_test_pred_proba

array([[0.97435831, 0.02564169],
       [0.41691179, 0.58308821],
       [0.6105662 , 0.3894338 ],
       ...,
       [0.98495995, 0.01504005],
       [0.66787784, 0.33212216],
       [0.20134248, 0.79865752]])

### Métricas generales

In [77]:
accuracy_score(y_test, y_test_pred_1)

0.7931034482758621

In [78]:
model_1.intercept_

array([-0.26223816])

In [79]:
model_1.coef_

array([[-5.88935272e-02,  6.79101730e-03,  2.42911969e-04,
        -3.07966559e-02,  1.71577093e-01, -5.59663480e-04,
        -1.38845379e-01, -6.19481208e-01,  3.57243052e-01,
         2.87780114e-01,  7.81634638e-01, -6.91957234e-02,
        -6.91957234e-02, -4.68684075e-01, -6.91957234e-02,
        -2.39237856e-01, -6.91957234e-02, -1.22103470e-01,
        -6.91957234e-02, -4.07764121e-01, -6.91957234e-02,
         1.51987866e-01, -6.91957234e-02,  1.70724201e-01,
        -5.23366555e-01, -8.34416408e-01,  3.34498620e-01,
        -1.26254540e-01,  2.79533876e-01,  1.85891691e-03]])

Matriz de confusión

In [80]:
confusion = confusion_matrix(y_test, y_test_pred_1)

TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

In [81]:
print('TP: ', TP,' TN: ', TN,' FP: ',FP,' FN: ',FN )
print('Accuracy=', (TP+TN)/ (TP+TN+FP+FN))
print('Total de casos correctamente predichos (TP+TN) =',(TP+TN))
print('Total de casos (TP+TN+FP+FN) =',(TP+TN+FP+FN))

TP:  209  TN:  987  FP:  121  FN:  191
Accuracy= 0.7931034482758621
Total de casos correctamente predichos (TP+TN) = 1196
Total de casos (TP+TN+FP+FN) = 1508


### Métricas específicas

**RECALL:** Si su valor es bajo, es porque hay presencia de falsos negativos. Por eso, esta medida es sensible a los FN.

Comparado con accuracy_score, esta medida se enfoca en los casos positivos, así muestra cómo funciona nuestro modelo en relación al objeto de interés de nuestro negocio.

Útil cuando la ocurrencia de falsos negativos es inaceptables.

In [82]:
recall_score(y_test,y_test_pred, pos_label='No')

0.8907942238267148

**PRECISION:** Si su valor es bajo, es porque hay presencia de falsos positivos. Por eso, esta medida es sensible a los FP.

Util cuando necesitamos estar seguros de los verdaderos positivos.

In [83]:
precision_score(y_test, y_test_pred_1, pos_label='No')

0.8378607809847198

**Specificity:** (especificidad o true negative rate (TNR)) es la proporción de negativos correctamente predichos sobre el total de casos negativos.

Mide qué tan "específico" es el clasificador al predecir las instancias positivas. Se calcula como el número de verdaderos negativos (TN) sobre todos los casos que son negativos (TN+FP).

Si su valor es bajo, es porque hay presencia de falsos positivos. Por eso, esta medida es sensible a los FP.

Otro ejemplo donde importa una alta especificidad, es si predecimos que una persona está enferma al cual debemos suministrarle una droga potente, y no lo está realmente.

In [84]:
print('Specificity=', (TN)/ (TN+FP))
print('Total de casos negativos predichos correctamente (TN) =',(TN))
print('Total de casos negativos (TN+FP) =',(TN+FP))

Specificity= 0.8907942238267148
Total de casos negativos predichos correctamente (TN) = 987
Total de casos negativos (TN+FP) = 1108


**F1-Score:** Como regla general, cuanto mayor es esta métrica, mejor es el modelo.

Pero para tener un f1-score alto, es necesario que tanto recall como precision sean altos, mientras que un f1-score bajo puede ser el resultado de un valor bajo en por lo menos una de estas métricas o en ambas a la vez.

In [85]:
f1_score(y_test,y_test_pred_1, pos_label='No')

0.8635170603674541

La ventaja de usar la media armónica (en vez de la media aritmética) es que el resultado del f1-score no es sensible a valores altos de una de las dos variables (recall o precision).

Por otro lado, no todos los valores extremos son ignorados, ya que los que son muy bajos si tienen peso en el resultado final.

<hr/>

### IMPORTANTE

* Debemos tener en cuenta Recall cuando no podemos aceptar los falsos negativos.
* Specificity cuando no debemos aceptar falsos positivos.
* Precision cuando debemos estar seguros de los verdaderos positivos.

**Cuál es la pregunta de negocio? Qué métrica se quiere maximizar?**
En base a esto decidiremos cuales son las mejoras a aplicar en los siguientes pasos.

<hr/>

In [ ]:
model_2 = LogisticRegression(penalty='L1')
model_2.fit(X_train, y_train);

In [86]:
y_test_pred_2 = model_1.predict(X_test)
y_test_pred_2

array(['No', 'Yes', 'No', ..., 'No', 'No', 'Yes'], dtype=object)

In [87]:
f1_score(y_test,y_test_pred_2, pos_label='No')

0.8635170603674541